In [2]:

using POMDPs, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies, POMDPModelTools
using Distributions, Combinatorics, StaticArrays, Statistics
using FileIO, JLD2, TickTock

[ Info: Precompiling POMDPModelTools [08074719-1b2a-587c-a292-00f91cc44415]
┌ Warning: Module POMDPTools with build ID fafbfcfd-fe8b-272a-0002-4d2f389f2201 is missing from the cache.
│ This may mean POMDPTools [7588e00f-9cae-40de-98dc-e0c70c48cdd7] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing POMDPModelTools [08074719-1b2a-587c-a292-00f91cc44415].


## State Functions

In [3]:
function state_cnt(n, S) #n = number of units; S = number of states
    if S==1
        return 1
    end
    return binomial(n+S-1, S-1)
end

function state_index(S,s) #s = state vector
    #S = length(s)
    if S == 1
        return 1
    end
    if s[S]==0
        return state_index(S-1, s[1:(S-1)])
    end
    
    n_prev = sum(s)
    prev = state_cnt(n_prev, S-1) # count of all states with s[S]=0
    inc = prev
    for i in 1:(s[S]-1)
        inc = inc/(n_prev+S-2)*n_prev #count of all states with s[S]=i
        prev = prev + inc
        n_prev = n_prev - 1
    end
    return prev + state_index(S-1, s[1:(S-1)])
end

function state_vec(n, S, ind) # n = number of units; S = number of states; ind = state index
    if ind < 0.5 || ind > state_cnt(n, S) + 0.5
        println("index is out of range!")
        return -1
    end
    if S==1 
        return [n]
    end
    if ind < state_cnt(n, S-1) + 0.5
        return push!(state_vec(n, S-1, ind), 0)
    end
    prev = state_cnt(n, S-1)
    inc = prev
    last_state = 0
    n_prev = n
    while ind > prev + 0.5
        inc = inc/(n_prev+S-2)*n_prev #count of all sta_tes with s[S]=last_state
        prev += inc
        n_prev -= 1
        last_state += 1
    end
    return push!(state_vec(n-last_state, S-1, ind - prev + inc), last_state)
end

#using hueristic rule for rollout
mutable struct nNRollout <: Policy
    n::Int64
    N::Int64
end

mutable struct nmNRollout <: Policy
    n::Int64
    m::Int64
    N::Int64
end

mutable struct mystate
    state::Vector{Int64};
end

In [4]:
function POMDPs.action(p::nNRollout, s::mystate)
        local_a = 0;
        local_s = s.state;
        nN = sample(1:Number_level, 2, replace = true);
        p.N = maximum(nN);
        p.n = minimum(nN);
        if sum(local_s[p.N:Number_level])>=1
        local_a = p.n;
        end
        return local_a; 
end

function POMDPs.action(p::nmNRollout, s::mystate)
    local_a = 0;
    local_s = s.state;
    nmN = sample(1:Number_level, 3, replace = true);
    p.N = maximum(nmN);
    p.n = minimum(nmN);
    p.m = sum(nmN)-p.N-p.n;
    if sum(local_s[p.N:Number_level])>=1 || sum(local_s[p.m:Number_level])>=2
        local_a = p.n;
    end
    return local_a; 
end

In [5]:
function findNn()
    println("Finding best nN policy...")

for N in 2:(Number_level)
    for n in 1:N
#         println("N = ",N, "  n = ",n);
        Threads.@threads  for j in 1:repetition
            history = sim(multiunit2,max_steps=simsteps,initialstate=mystate(state_vec(NumberUnits, Number_level, 1))) do s
            local_s = s.state;
            local_a = 0;
                if sum(local_s[N:Number_level])>=1 
                local_a = n;
                end
                return local_a; 
            end
            temp_rewards[j] = discounted_reward(history);
        end
        rewards_nN[n,N] = mean(temp_rewards);
        rewards_nN_std[n,N] = std(temp_rewards);
        
    end
end
(max_rward,nN) = findmax(rewards_nN);
println("Max rewards of (N, n) policy is ", max_rward, "  n is ", nN[1], "  N is ",nN[2]);
return  max_rward,rewards_nN_std[nN] ,nN;
end


findNn (generic function with 1 method)

In [6]:
using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],mean=[],std=[],nN=[])

cost=[
    [0	-600	-100	-1200 -2000],
    [0	-800	-100	-1200 -2000],
    [0	-1000	-100	-1200 -2000],
    [0	-1400	-100	-1200 -2000],
    [0	-1800	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000]
] 

for units in [150,140,130,120,110,100,90,80,70,60,50,40,30,20]
    for K in 2:convert(Int64, floor((units+1)/10)):convert(Int64, floor((units+1)/2))
        
    global NumberUnits=convert(Int64,units)
        global limit=convert(Int64,K)

#action_level = 0;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
#fullname = "C:/Users/yongchen/OneDrive - University of Iowa/Codes/Julia/ReinforcementLearning/tm6.jld2"
#fullname = "C:/Users/yongchen/Dropbox/reinforcement learning for maintenance/Matlab/MultipleUnits/tm6.jld2";
global Number_level = 4;
global Transition_matrix = [
    0.8571 0.1429 0.0    0.0;
    0.0    0.8571 0.1429 0.0;
    0.0    0.0    0.8    0.2;
    0.0    0.0    0.0    1.0
]



global state_number = state_cnt(NumberUnits,Number_level);
# global arrayofstates = Vector{mystate}(undef, state_number);

# Threads.@threads for i in 1:state_number
#     global arrayofstates[i]  = mystate(state_vec(NumberUnits, Number_level, i));
# end

global crd = Array{Categorical}(undef,Number_level)
for i in 1:Number_level
    global crd[i] = Categorical(Transition_matrix[i,:]);
end
    
    
    
global multiunit2 = QuickMDP(
    gen = function (s, a, rng)       #s is a vector of number units at each level and a is the number of units we will repair
        local_s = s.state;
        # println(local_s)
        # based on s, create a status vector corresponding to each unit
        degradation_state = repeat(1:1,NumberUnits);
        k = 1;
        for i in 1:Number_level
            for j in 1:local_s[i]
                degradation_state[k]=i;
                k = k+1;
            end
        end
        r = 0.0;
        number_reset=0
        prevent_repair = false;
        if a!=0
        number_reset = sum(local_s[a:Number_level]);
        end
        #using a for loop to compute next state for each unit
        if local_s[Number_level] >= limit && a==0
                        r+=system_penalty
                    number_reset=local_s[Number_level]
                    end 
        if local_s[Number_level] >= limit && a!=0
                        r+=system_penalty
                     number_reset = sum(local_s[a:Number_level]);
                    end 
        for i in 1:(NumberUnits-number_reset)  #a is the number of units we want to preventively repair
            #in this loop, all units continues
                    if degradation_state[i] != Number_level
            degradation_state[i] = rand(crd[degradation_state[i]]);
            r = r+normal_operation;
                    end
            # if degradation_state[i] == Number_level
            #     r+=system_penalty/limit
            # end
        end
        
        for i in (NumberUnits-number_reset+1):NumberUnits
                if degradation_state[i] == Number_level
                    r = r + failure_penalty  ;
                if prevent_repair == false
                    r = r+setup_cost;
                    prevent_repair = true;
                            end    
                    else
                    r = r + maintenance_penalty;
                if prevent_repair == false
                    r = r+setup_cost;
                    prevent_repair = true;
                end
                    end
            degradation_state[i] = rand(crd[1]); #reset status; add additional transition
            r = r+normal_operation; #add operation benefit
         end
        #collect degradation state to form the state
        sp = repeat(0:0,Number_level);
        for i in 1:NumberUnits
            sp[degradation_state[i]] = sp[degradation_state[i]]+1;
        end
        return (sp=mystate(sp), r=r)
    end,
    actions = 0:(Number_level-1), 
    actiontype = function()
        return Int64;
    end,
#     states = arrayofstates,
    initialstate = function()
        POMDPModelTools.ImplicitDistribution() do rng
            return (mystate(state_vec(NumberUnits, Number_level, 1)))
        end
    end, #all u #all units start fresh. Need to change according to unit number and level number. ##For simulation, we need to use ImplicitDistribution
    discount = 0.95,
    isterminal = false              # no ending
    )


    
    
    
for c in cost
        tick()
   println(units)  
             print("K : ")
                    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println(c)     
   global  simsteps = 100;
global repetition = 10000;
global rewards_nN = zeros(Number_level,Number_level);
global rewards_nN = rewards_nN.+(-100000000.0);
global rewards_nN_std = zeros(Number_level,Number_level);
global rewards_nN_std = rewards_nN_std.+(-100000000.0);

global discount_factor =0.95
global temp_rewards = zeros(repetition,1);
rewards,std, nN=findNn()
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,rewards,std, nN])
        tock()   
    end
 
    
        end   
    end

150
K : 2


[ Info:  started timer at: 2024-03-27T21:55:54.426


[0 -600 -100 -1200 -2000]
Finding best nN policy...
Max rewards of (N, n) policy is -27799.082500539833  n is 3  N is 3
150
K : 2
[0 -800 -100 -1200 -2000]
Finding best nN policy...


[ Info:          195.8733447s: 3 minutes, 15 seconds, 873 milliseconds
[ Info:  started timer at: 2024-03-27T21:59:10.444


Max rewards of (N, n) policy is -31382.818446455003  n is 3  N is 3
150
K : 2
[0 -1000 -100 -1200 -2000]
Finding best nN policy...


...

[ Info:             46.18179s: 46 seconds, 181 milliseconds
[ Info:  started timer at: 2024-03-28T11:37:28.507


Max rewards of (N, n) policy is -12520.118643904092  n is 2  N is 4
20
K : 10
[0 -2400 -100 -1200 -2000]
Finding best nN policy...


[ Info:           46.9309805s: 46 seconds, 930 milliseconds
[ Info:  started timer at: 2024-03-28T11:38:15.461


Max rewards of (N, n) policy is -14284.614387567794  n is 2  N is 4


[ Info:           47.0243495s: 47 seconds, 24 milliseconds


In [7]:
df

Row,u,K,n,s,m,f,p,mean,std,nN
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,20,10,0,-2400,-100,-1200,-2000,-14284.6,1678.47,"CartesianIndex(2, 4)"
2,20,10,0,-1800,-100,-1200,-2000,-12520.1,1471.75,"CartesianIndex(2, 4)"
3,20,10,0,-1400,-100,-1200,-2000,-11355.3,1320.66,"CartesianIndex(2, 4)"
4,20,10,0,-1000,-100,-1200,-2000,-10027.6,935.315,"CartesianIndex(2, 3)"
5,20,10,0,-800,-100,-1200,-2000,-8878.18,808.533,"CartesianIndex(2, 3)"
6,20,10,0,-600,-100,-1200,-2000,-7740.55,693.263,"CartesianIndex(2, 3)"
7,20,8,0,-2400,-100,-1200,-2000,-14288.3,1686.6,"CartesianIndex(2, 4)"
8,20,8,0,-1800,-100,-1200,-2000,-12541.0,1477.94,"CartesianIndex(2, 4)"
9,20,8,0,-1400,-100,-1200,-2000,-11386.2,1354.53,"CartesianIndex(2, 4)"


In [9]:
using CSV
CSV.write("nN_simple(case study).csv", df)

"nN_simple(case study).csv"

In [ ]:
#= using Plots
 =#

In [ ]:
#= plot(df[df.s.==-250,:].u,df[df.s.==-250,:].mean) =#

In [ ]:
#= convert(Int64,df.mean[1]) =#

In [ ]:
#= plot(df.u,df.mean.+1100.0,yticks=df.mean,xticks=df.u,ytickfontrotation=10.0)
scatter!(df.u,df.mean.+1100.0)
plot!(xscale= :identity
    
    ,yscale=:log10,ytickfontrotation=10.0)
 =#

In [ ]:
#= plot(df[df.s.==-300,:].u,df[df.s.==-300,:].mean,label="nN") =#